In [ ]:
LAB_USER_NAME = 'DEMO'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [ ]:
import panel as pn
pn.extension(raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import sys
import os
import numpy as np

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

### Lab 2 structure:

#### After this lab, the student should have learned (optimistically):
- **How the precession frequency at different locations is affected during a magnetic field gradient**
  - The precession frequency will have an offset proportional to the coordinate along the gradient axis
- **How the precession phase at different locations is affected after a gradient pulse**
  - The precession phase will be shifted by an amount proportional to the area under the gradient pulse and the coordinate along the gradient axis
- **How frequency encoding allows position to be measured**
  - By taking the fourier transform of the signal acquired with a gradient active, the coordinate along gradient axis can be calculated from the frequency offset and gradient amplitude
- **How phase encoding allows position to be measured**
  - Applying a gradient pulse before acquisition allows measuring a single spatial frequency.
    Doing this multiple times with different gradient areas allows measuring a range of spatial frequencies, which can be used to reconstruct the image with a fourier transform.
- **How the k-space spectrum relates to the image**
  - K-space is the fourier transform of cartesian coordinate space. Coordinates in k-space are spatial frequencies, and the value at a coordinate tells you how well the image matches a sinewave with that wavelength (inverse of spatial frequency) and direction.
- **How the k-space coordinate of a discrete signal sample is determined from the gradient pulse sequence**
  - Each k-space coordinate of a sample is proportional to the area under the gradient applied along that coordinate axis between the excitation (RF 90 pulse) and the acquisition of that sample.
- **What (in k-space, and in the pulse sequence) determines the field of view/zoom of the image**
  - Field of View is determined by the spacing between samples in k-space, which is determined by the difference in area under the gradient pulses of two successive samples.
    A larger spacing results in a smaller field of view.
    For frequency encoding the k-space spacing is the gradient amplitude multiplied by the dwell time, and for phase encoding it is the gradient step size multiplied by the phase gradient duration.
- **What (in k-space, and in the pulse sequence) determines the spatial resolution of the image**
  - Spatial resolution is determined by the extent of the k-space spectrum. 
    Sampling a larger region of k-space results in a smaller (in meters, i.e. better) spatial resolution. 
    For frequency encoding the size of k-space is the gradient amplitude multiplied by the total acquisition time, and for phase encoding it is the area under the maximum gradient pulse.

#### Diagrams:
1. SE with constant shim
2. SE with Z frequency encode
3. SE with XY vector freq encode
4. SE with X frequency and Y phase encode gradients
5. ~~SE with RF soft pulse and constant shim gradient~~
6. ~~Full slice 2D SE~~

#### Samples:
1. Uniform sample (shim sample)
2. Single thin layer sample
3. 1D Z phantom with multiple alternating layers of plastic and water (e.g. 5 plastic, 4 water, height: 20mm plenty of plastic padding above and below to exclude surrounding water)
4. XY projection phantom with two vertical water cylinders in plastic
5. ~~Slice phantom?~~

#### Experiments:
1. Full acquisition spin echo with adjustable x,y,z shims.
2. 1D SE with Z frequency encode. Allow altering the gradient strength and observing the resulting signal and spectrum.
3. 1D SE with X/Y oblique frequency encode. Allow altering gradient magnitude and angle and observing the resulting signal and spectrum.
4. 2D XY SE with X frequency encode, Y phase encode. Allow adjusting X and Y gradient strengths.
5. ~~Full acquisition spin echo with adjustable width soft pulse and adjustable x,y,z shims.~~
6. ~~2D SE with slicing~~

#### Steps:
1. See how X, Y, Z shims alter the spin echo signal and spectrum, with uniform sample and 1D Z phantom
2. Run 1D SE experiment
3. Run a 2D XY SE with X frequency encode, Y phase encode.
    - Allow adjusting X and Y gradient strengths and number of samples/steps. Display Y gradient step size.
    - Display k-space and the image, updating after every acquisition so they can see how the image looks as k-space is built up.
    - Fix the kx,ky,x,y axes so they can see the size of the k-space sampling area and the image FOV changing
4. 


## 1. Gradients

Part of Lab 1 involved optimising the duration of the NMR signal by correcting the magnetic field inhomogeneities (unevenness) using shims. Inhomogeneity caused nuclei at different locations in the sample to resonate at slightly different frequencies, causing destructive interference and reduction in signal. The fundamental principle of NMR imaging is to exploit the effect of inhomogeneity to locate the source of an NMR signal in space. To purposely distort the field in a way that is simple to analyse, we will use combinations of coils that create linear **gradients** in the the field strength along the $x$, $y$, and $z$ axes.

Illustrations of these coils are shown in Figure 1.1 on the right (TODO).

Because the gradients are linear, the Larmor frequency at any location will be a linear function of its coordinate along the gradient axis, e.g. when applying gradient $G_z$:

$$ f = f_0 + \bar\gamma \cdot G_z \cdot z \tag{1.1}$$

Where $f_0$ is the Larmor frequency when no gradient is applied, and $\bar\gamma$ is the gyromagnetic ratio ($\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$).

In the next task we will start with the gradients set to the optimised shim values, and then modify them to see how they can be used to encode some information about the position of the sample into the signal.

TODO diagrams:
- X/Y/Z gradient coil renders/3d diagrams
- Bore field strength with z gradient
- graph for of freq vs z position with z gradient 


#### **Task 1.1**

1. Insert the thin layer sample (make sure to centre the sample using the depth gauge).
2. Reset shims and start experiment with "Run Loop"
3. Zoom in on the echo signal (from 6us to 14us)
4. Move the sample up and down. Q: What happens to the signal when it is moved too far from the centre?
5. Return the sample to roughly the centre (where the signal returns to full amplitude)
6. Increase the Z shim to its limit (1.0, or -1.0). Q: How does the signal shape change when the shim is offset?
7. Move the sample up and down a small amount, so that the magnitude is not significantly affected. Q: What happens to the signal frequency?
8. Click abort to stop the experiment before moving on

In [ ]:
# Experiment 1: Full acquisition SE with constant shim
# load global shims file
import yaml
from matipo import GLOBALS_DIR
SHIM_ORDER = ['shim_x', 'shim_y', 'shim_z']
SHIM_FILE = os.path.join(GLOBALS_DIR, 'shims.yaml')

with open(SHIM_FILE, 'r') as f:
    SHIM_INIT = yaml.safe_load(f)

# create shim inputs, using saved user shims as initial values
shim_inputs = {}
for shim_key in SHIM_ORDER:
    shim_name = shim_key.split('_')[-1].upper()
    shim_inputs[shim_key] = pn.widgets.FloatInput(name=shim_name+' shim', start=-1, end=1, step=0.01, value=round(SHIM_INIT[shim_key], 2), width=80)

# button to reset shim inputs to the saved values
reset_btn = pn.widgets.Button(name='Reset', button_type='primary', align='end', width=100)

def reset_shim_inputs(e):
    for shim_key in SHIM_ORDER:
        shim_inputs[shim_key].value = round(SHIM_INIT[shim_key], 2)

reset_btn.on_click(reset_shim_inputs)
    
from full_acq_SE import FullAcqSEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    t_echo=0.01,
    n_scans=1,
    n_samples=1500,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# add shim inputs
override_pars.update(shim_inputs)

# create dashboard app
exp1_app = FullAcqSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True
)

# display layout
pn.Column(
    # echo_time,
    pn.Row(*([shim_inputs[shim_key] for shim_key in SHIM_ORDER]+[reset_btn])), # take shim inputs in order and concatenate reset button
    exp1_app.main(),
    sizing_mode='stretch_width'
)

## 2. Frequency Encoding

The experiments in **Task 1.1** showed how the frequency of the NMR signal from a small sample varies with position when a gradient is applied by offsetting a shim value. In fact, with calibration to determine the value of $G_z$, the frequency could be measured and the position calculated by rearranging Equation 1.1.

Now consider a sample made up of multiple small partitions of fluid; each partition will emit a signal with frequency depending on its position, but the measured signal will be the sum of all of them, a complex shape caused by constructive and destructive intereference. The frequencies present in the measured signal can be extracted using a Fourier Transform, revealing individual peaks associated with the partitions (See Figure 2.1).

In fact for any shape of sample in a field with a linear gradient, you can imagine slicing it along the gradient axis, summing the signals from the slices, and taking the Fourier Transform to obtain a measurement of the amount of nuclei in each slice. This results in a 1D projection image of the sample.

Three important characteristics of this image are:
- **Field of View (FOV)**: The physical size of the image, i.e. the size of the largest object that can be viewed without being cropped or wrapped.
- **Spatial Resolution**: The level of detail, usually expressed as a minimum distance between features such that they are still resolved, and not merging into a single blob. Note that this is not the same as the number of pixels in the image (Pixel Resolution). e.g. a spatial resolution of 1 mm is *better resolution* than a spatial resolution of 2 mm.
- **Signal to Noise Ratio (SNR)**: The ratio of the amplitude of the signal in areas of image corresponding to the nuclei in the sample to the amplitude of the noise in areas without signal. A larger SNR is better.

For simplicity, the shim values and gradient values are generally set seperately and added together by the hardware. The gradient also does not need to be active for the entire pulse sequence, only during the acquisition time and for a short time between the $90^\circ$ and $180^\circ$ RF pulses – to satisfy a requirement for optimal spin echo formation where the area under the gradient between the $90^\circ$ and $180^\circ$ pulses must match the area under the gradient from the $180^\circ$ to the echo time ($T_E$). This pulse sequence is shown in Figure ? below (with the shim values hidden).

Diagrams:
- pulse sequences: se with constant shim, se with read gradient pulses
- diagrams of FOV and spatial resolution?
- if unclear, sample slicing and resulting spectrum diagram?

#### **Task 2.1**
1. With the thin layer sample still inserted, start the experiment with "Run Loop"
2. Increase the gradient to 0.25.
3. Move the sample up and down again, and see how the spectrum changes. Q: Does this match your observations in the previous experiment?
4. Inspect the 1D Z phantom and, based on your observations with the thin layer sample, draw a rough plot of the spectrum you would expect to see when it is inserted.
5. Insert the 1D Z phantom (centred with the depth gauge). Q: Does the spectrum look as you expected?
6. Adjust the gradient strength through its possible range and note how the spectrum "image" changes. Q: Comparing the image with 0.25 gradient and 0.5 gradient: Which has a larger field of view (FOV)? Which has better signal-to-noise ratio (SNR)? Which has better spatial resolution?
7. Click abort to stop the experiment before moving on

TODO: allow modifying dwell time and number of samples?

In [ ]:
# Experiment 2: SE with adjustable Z read gradient showing signal and spectrum of echo
# read gradient input
# TODO: calibrate and use mT/m
read_z_grad = pn.widgets.FloatInput(name='Read Z Gradient', start=-1, end=1, step=0.01, value=0, width=200)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
from SE import SEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (0, 0, read_z_grad.value),
    t_echo=0.012,
    n_scans=1,
    n_samples=1000,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# create dashboard app
exp2_app = SEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True
)

# display layout
pn.Column(
    read_z_grad,
    exp2_app.main(),
    sizing_mode='stretch_width'
)

## 3. Projection Reconstruction

With the above method of creating a 1D projection image, a 2D image can be reconstructed by combining multiple images with the sample rotated at different angles. This is the method used to produce 3D images from X-ray projections, called Computed Tomography (with the X-ray machine rotated around the human). If we apply a combination of linear gradients they will add together to produce a net gradient in an oblique (off-axis) direction. Using this method the gradient can be rotated instead of the rotating the sample.

In the next experiment, the read gradient will be specified with a magnitude $|G|$ and angle $\theta$, and the $x$ and $y$ components will be automatically calculated using:

$$
G_x = |G| \cdot \cos \theta \\
G_y = |G| \cdot \sin \theta
\tag{3.1}
$$

Diagrams
- projection reconstruction example

#### **Task 3.1**
1. Insert the mystery sample at the correct depth and start the experiment below with "Run Loop"
2. Increase the magnitude to expand the 1D image and try both physically rotating the sample and rotating the read gradient direction.
3. Set the gradient magnitude appropriately so that the 1D image is not cropped with any gradient direction.
4. Based on 1D views at different angles, draw a 2D top-down diagram of where you think the water is located in the mystery sample. Hint: draw a square in your notebook, and for each edge draw the 1D image you get from orienting the gradient along that edge. Based on this you can draw inside the square where the water is located.
5. Click abort to stop the experiment before moving on

In [ ]:
# Experiment 3: 1D SE with adjustable X/Y vector read gradient
# read gradient input
# TODO: calibrate and use mT/m
exp3_read_grad_mag = pn.widgets.FloatInput(name='Read Gradient Magnitude', start=0, end=1, step=0.01, value=0, width=200)
exp3_read_grad_dir = pn.widgets.FloatInput(name='Read Gradient Direction (degrees)', start=0, end=360, step=10, value=0, width=200)
exp3_read_x_grad = pn.widgets.FloatInput(name='Read Gradient X Component', start=-1, end=1, step=0.01, value=0, width=200, disabled=True)
exp3_read_y_grad = pn.widgets.FloatInput(name='Read Gradient Y Component', start=-1, end=1, step=0.01, value=0, width=200, disabled=True)

def exp3_read_grad_cb(*events):
    exp3_read_x_grad.value = exp3_read_grad_mag.value*np.cos((np.pi/180)*exp3_read_grad_dir.value)
    exp3_read_y_grad.value = exp3_read_grad_mag.value*np.sin((np.pi/180)*exp3_read_grad_dir.value)

exp3_read_grad_mag.param.watch(exp3_read_grad_cb, ['value'], onlychanged=True)
exp3_read_grad_dir.param.watch(exp3_read_grad_cb, ['value'], onlychanged=True)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
from SE import SEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (exp3_read_grad_mag.value*np.cos((np.pi/180)*exp3_read_grad_dir.value), exp3_read_grad_mag.value*np.sin((np.pi/180)*exp3_read_grad_dir.value), 0),
    t_echo=0.012,
    n_scans=2,
    n_samples=1000,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.1
)

# create dashboard app
exp3_app = SEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=False,
    enable_run_loop=True
)

# display layout
pn.Column(
    pn.Row(exp3_read_grad_mag, exp3_read_grad_dir, exp3_read_x_grad, exp3_read_y_grad),
    exp3_app.main(),
    sizing_mode='stretch_width'
)

## 4. *k*-space

The signal we acquire must generally be Fourier transformed to produce an image, and in order to understand how the pulse sequence and acquisition samples relate to the image obtained it is helpful to use the concept of *k*-space. A *k*-space image is the inverse Fourier transform of the image in physical space (the "normal" image). The value of a pixel in a *k*-space image represents the amount of a particular *spatial frequency* (units of radians per meter) in the physical space image, with the centre of *k*-space corresponding to low spatial frequencies and the edges to high spatial frequencies.

Every discrete sample acquired during the pulse sequences has a *k*-space coordinate depending on the gradients and RF pulses applied so far. For frequency encoding as used in **Section 2**, $k_z$ evolves as shown in Figure 4.1 (A). The $180^\circ$ RF pulse inverts the accumulated phase of nuclei that were excited by the initial $90^\circ$ pulse, which also inverts the *k*-space coordinate. The pulse sequences is designed so that at the beginning of sampling $k_z$ is negative and passes through zero at $T_E$ before becoming positive. In this way we sample along a line segment through *k*-space centred at $k_z=0$, see Figure 4.1 (B).

Sampling along a line segment in *k*-space produces a 1D image projected onto a line segment in physical space, and in **Section 3** it was possible to build a rough 2D image by combining multiple 1D projections. In *k*-space, the data acquired lies on line segments through the centre of 2D k-space as shown in Figure 4.2. In principle, instead of processing each line individually with a 1D Fourier transform, we can generalise and apply a 2D Fourier transform to our 2D *k*-space data. In practice this is difficult to do with the data from these line segments because they do not fall in a regular grid, and the Fast Fourier Transform algorithm cannot be used directly. There are advanced data processing techniques that can resample the data into a grid, but these will not be covered in this lab.

Instead, we will modify the pulse sequence to cause the *k*-space coordinates to fall into a regular grid. This is done using *phase encoding*, where a gradient pulse is applied before sampling, causing the accumulated phase of precession to change depending on position. After a $y$ gradient pulse with amplitude $G_y$ and duration $t_y$ the final phase at coordinate $y$ is:

$$ \phi = \bar\gamma \cdot G_y t_y \cdot y$$

In this way, the $y$ position could be determined from the phase of the signal of a small partition of sample. With a large sample the signals from different $y$ locations will sum together into a single phase and magnitude, so multiple acquisitions stepping through values of $G_y t_y$ are required to fill the *k*-space grid as shown in Figure 4.3.

- Figure 4.1 (A): k_z vs time for 1D SE experiment
- Figure 4.1 (B): k-space trajectory in 1D (z)
- Figure 4.2: multiple lines at different angles through origin in k-space (k_x, k_y)
- Figure 4.3: 2D SE with all phase steps, *k*-space diagram with samples in grid and a few example trajectories drawn.

## 5. 2D Imaging

diagrams:
- read encoding SE pulse sequence and k-space trajectory for a read gradient vector rotated 30 degrees ccw from the x axis
- phase/read encoding SE pulse sequence and k-space trajectory
- phase encoding spin diagram
- phase wrapping image


Phantoms:
- make cube phantom that allows them to make sure the image has the right proportions

#### **Task 5.1**
- TODO

In [ ]:
# Experiment 4: 2D SE with adjustable X read gradient and Y phase gradient showing kspace and image
read_x_grad = pn.widgets.FloatInput(name='Read X Gradient', start=-1, end=1, step=0.01, value=0, width=200)
samples = pn.widgets.IntInput(name='Read Samples', start=1, end=200, step=1, value=32, width=200)
dwell_time = pn.widgets.IntInput(name='Dwell Time (us)', start=2, end=80, step=2, value=20, width=200)
phase_y_grad = pn.widgets.FloatInput(name='Phase Y Gradient', start=-1, end=1, step=0.01, value=0, width=200)
phase_steps = pn.widgets.IntInput(name='Phase Steps', start=1, end=200, step=1, value=32, width=200)
phase_y_width = pn.widgets.IntInput(name='Duration (us)', start=10, end=1000, step=10, value=200, width=200)

def get_phase_order():
    g_phase_prop = np.linspace(1, -1, phase_steps.value, endpoint=False)
    phase_order = np.argsort(np.abs(g_phase_prop-1e-6))
    return phase_order

def get_g_phase():
    g_phase_max = phase_y_grad.value
    g_phase_prop = np.linspace(1, -1, phase_steps.value, endpoint=False)
    return np.outer(phase_y_grad.value*g_phase_prop[get_phase_order()], np.array([0, 1, 0]))


import importlib
import RARE2D
importlib.reload(RARE2D)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
# TODO: fix phase order so it is consistent
# TODO: change k-space colormap
from RARE2D import RARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (read_x_grad.value, 0, 0),
    g_phase_1=get_g_phase,
    phase_order=get_phase_order,
    n_ETL=1,
    t_echo=0.01,
    n_scans=1,
    n_samples=samples,
    t_dw=lambda: 1e-6*dwell_time.value,
    t_read=lambda: 1e-6*dwell_time.value*samples.value,
    t_phase=lambda: 1e-6*phase_y_width.value,
    t_end=1,
)

# create dashboard app
exp4_app = RARE2DApp(
    override_pars=override_pars
)

# display layout
pn.Column(
    pn.Row(read_x_grad, samples, dwell_time),
    pn.Row(phase_y_grad, phase_steps, phase_y_width),
    exp4_app.main(),
    sizing_mode='stretch_width'
)

In [ ]:
# Experiment ?: Full acquisition SE with sinc RF pulses and adjustable X/Y/Z shims

In [ ]:
# Experiment ?: 2D SE with slicing